In [7]:
#Importamos las librerias a utilizar. 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import dash
from dash import html
from dash import dcc
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output

import warnings
warnings.filterwarnings("ignore")

In [8]:
#Leemos la data.
importaciones = pd.read_csv("data_importaciones.csv", sep = ";")
consumo = pd.read_csv("data_consumo.csv", sep = ";")

In [9]:
#Renombramos las columnas.
data_con = consumo.rename(columns = {"GasolinaRegular": "Regular", "GasolinaSuper": "Super"})
data_imp = importaciones.rename(columns = {"GasolinaRegular": "Regular", "GasolinaSuper": "Super"})

In [10]:
#Vemos la cantidad de datos. 
print(len(data_con), len(data_imp))

252 252


In [11]:
#Vemos los nombres de las columnas de la data de consumo. 
data_con.columns

Index(['Anio', 'Mes', 'Super', 'Regular', 'Diesel', 'Fechas', 'Super_Arima',
       'Regular_Arima', 'Diesel_Arima', 'Super_SARIMAX', 'Regular_SARIMAX',
       'Diesel_SARIMAX', 'Super_EXP', 'Regular_EXP', 'Diesel_EXP'],
      dtype='object')

In [12]:
#Vemos los nombres de las columnas de la data de importacion.
data_imp.columns

Index(['Anio', 'Mes', 'Super', 'Regular', 'Diesel', 'Fechas', 'Super_Arima',
       'Regular_Arima', 'Diesel_Arima', 'Super_SARIMAX', 'Regular_SARIMAX',
       'Diesel_SARIMAX', 'Super_EXP', 'Regular_EXP', 'Diesel_EXP'],
      dtype='object')

### Bases para Consumo

In [13]:
#Quitamos las columnas que no se van a utilzar. 
RegularCon = data_con.drop(columns = ['Super','Diesel', 'Super_Arima', 'Diesel_Arima', 'Super_SARIMAX',
       'Diesel_SARIMAX', 'Super_EXP', 'Diesel_EXP'])

SuperCon = data_con.drop(columns = ['Regular', 'Diesel',
       'Regular_Arima', 'Diesel_Arima', 'Regular_SARIMAX',
       'Diesel_SARIMAX', 'Regular_EXP', 'Diesel_EXP'])

DieselCon = data_con.drop(columns = ['Super', 'Regular', 'Super_Arima',
       'Regular_Arima', 'Super_SARIMAX', 'Regular_SARIMAX', 'Super_EXP', 'Regular_EXP'])

In [14]:
#Vemos de nuevo los nombres de las columnas de la data de consumo.
RegularCon.columns

Index(['Anio', 'Mes', 'Regular', 'Fechas', 'Regular_Arima', 'Regular_SARIMAX',
       'Regular_EXP'],
      dtype='object')

In [15]:
#Renombramos las columnas. 
RegularCon = RegularCon.rename(columns = {"Regular_Arima": "Arima", "Regular_SARIMAX": "Sarimax", "Regular_EXP": "Exponencial"})
SuperCon = SuperCon.rename(columns = {"Super_Arima": "Arima", "Super_SARIMAX": "Sarimax", "Super_EXP": "Exponencial"})
DieselCon = DieselCon.rename(columns = {"Diesel_Arima": "Arima", "Diesel_SARIMAX": "Sarimax", "Diesel_EXP": "Exponencial"})

### Bases para importaciones

In [16]:
#Quitamos las columnas que no se van a utilzar. 
RegularImp = data_imp.drop(columns = ['Super','Diesel', 'Super_Arima', 'Diesel_Arima', 'Super_SARIMAX',
       'Diesel_SARIMAX', 'Super_EXP', 'Diesel_EXP'])

SuperImp = data_imp.drop(columns = ['Regular', 'Diesel',
       'Regular_Arima', 'Diesel_Arima', 'Regular_SARIMAX',
       'Diesel_SARIMAX', 'Regular_EXP', 'Diesel_EXP'])

DieselImp = data_imp.drop(columns = ['Super', 'Regular', 'Super_Arima',
       'Regular_Arima', 'Super_SARIMAX', 'Regular_SARIMAX', 'Super_EXP', 'Regular_EXP'])

In [17]:
#Renombramos las columnas. 
RegularImp = RegularImp.rename(columns = {"Regular_Arima": "Arima", "Regular_SARIMAX": "Sarimax", "Regular_EXP": "Exponencial"})
SuperImp = SuperImp.rename(columns = {"Super_Arima": "Arima", "Super_SARIMAX": "Sarimax", "Super_EXP": "Exponencial"})
DieselImp = DieselImp.rename(columns = {"Diesel_Arima": "Arima", "Diesel_SARIMAX": "Sarimax", "Diesel_EXP": "Exponencial"})

In [18]:
#Creacion del dashboard 
app = dash.Dash()

#Data a utilizar 
df = data_con
df2 = data_imp

#Titulo del Dashboard. 
app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Información sobre Gasolina', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40, 'color': 'teal'}),
    
    html.H2('Información Mensual', style = {'color': 'teal'}),
    html.P("Seleccione el tipo de Gasolina"),
    
        dcc.Dropdown(id = 'dropdown', #menu de opciones 
        options = [
            {'label':'Diesel', 'value':'Diesel' }, #lsita de opciones "label" el valor que se muestra en pantalla
            {'label': 'Regular', 'value':'Regular'}, #value es el valor del dropdown
            {'label': 'Super', 'value':'Super'},
            ],
        value = 'Diesel'), #valor inicial 
        html.P("Seleccione el rango de Meses"), #para texto
        dcc.RangeSlider(id = "range_slider", #Slider de rango (vector)
              min = 0, #el minimo
              max = len(df), #el máximo
              step = 1, #el tamaño de paso
              value = [0, len(df)], #el vector inciial
              marks = {str(i): str(i) for i in range(0, len(df), 12 )}), #valores que se muestran en pantalla
        dcc.Graph(id = 'bar_plot'),  #graficas en pantalla
        html.P('En el consumo mensual de gasolina regular puede observarse que existe un consumo relativamente \
               estable a lo largo de los meses. Puede verse un ligero aumento al final del primer trimestre del año, \
               para luego reducirse significativamente en el primer semestre del año. Luego, vuelve a crecer, aunque no \
               hay estabilidad en ello.'),
    
        html.P('Para el consumo mensual de gasolina super inicialmente se observa un cambio en el factor del eje y. Si bien \
               este se encuentra una decena por encima del anterior, al analizar los números, vemos que en realidad la cantidad\
               está cercana, pues pocos meses alcanza el 1.'),
    
        html.P('El consumo mensual de gasolina diesel es significativamente más fluctuante que los dos anteriores, así como\
               es más alto. Es lógico pensar que tenga una mayor popularidad, dado el transporte pesado, los servicios de transporte \
               y la fácil adquisición económica. En la gráfica es posible observar una caída por el mes 7, que luego empieza a crecer, \
               sin llegar a los valores del inicio de año.'),
    
        dcc.Graph(id = 'bar_plot_2'), #graficas
        
        html.P('En las importaciones vemos un comportamiento muy similar a al cosumo de gasolina mensual'),
    
        html.H2('Información Anual', style = {'color': 'teal'}),
        html.P("Seleccione el tipo de Gasolina"),
        dcc.Dropdown(id = 'dropdown_2',
        options = [
            {'label':'Diesel', 'value':'Diesel' },
            {'label': 'Regular', 'value':'Regular'},
            {'label': 'Super', 'value':'Super'},
            ],
        value = 'Diesel'),
        html.P("Seleccione el tipo de Gráfico"),
        dcc.Dropdown(id = 'dropdown_graphs',
        options = [
            {'label':'Bar Plot', 'value':'Bar' },
            {'label': 'Box Plot', 'value':'Boxplot'},
            ],
        value = 'Bar'),
    
        html.P("Seleccione el rango de Años"),
    
        dcc.RangeSlider(id = "range_slider_2",
              min = 1,
              max = len(df),
              step = 12,
              value = [1, len(df)],
              marks = {str(i): str( 2000 + int(i/12) ) for i in range(0, len(df), 12)}),
        dcc.Graph(id = 'bar_plot_3'),
        html.P('El comportamiento en la gasolina de consumo tiene un comportamiento bastante definido, y es que ha aumentado de forma casi estable a lo largo de todos los años. Se observa además un pico pronunciado en el consumo del año 2019, que sobresale ante los años anteriores. Sin embargo, vemos que el consumo para el año 2020 es significativamente menor en todos los casos, asemejándose sobremanera al comportamiento que se tuvo en 2012, 8 años antes. Dicha fluctuación puede explicarse para el 2020 por la pandemia de la COVID19, así como la clausura masiva que nuestra nación tuvo como medida de prevención.'),
    
        dcc.Graph(id = 'bar_plot_4'),
        
        html.P('La importación anual de gasolina regular tiene una importación baja en los primeros años y luego podemos observar como esta va aumentando, formando una gráfica con un sesgo moderado hacia la izquierda.'),
               
        html.P('En la grafica de importación anual de gasolina super podemos observar que inicia con una importación más alta que la gasolina regular, el cual va aumentando moderadamente formando un pequeño sesgo hacia la izquierda.'),
    
        html.P('En la importación anual de Diesel podemos observar que va en aumento e incluso en cuatro años se mantuvo casi constante y luego incremento significativamente.'),

        html.P('En estas tres gráficas podemos observar como el consumo disminuye en 2020 que es el año en el que todos nos vimos afectados por la pandemia'),
    
        html.H2('Información Mensual', style = {'color': 'teal'}),
        html.P("Seleccione el tipo de Gasolina"),
        dcc.Dropdown(id = 'dropdown_3',
        options = [
            {'label':'Diesel', 'value':'Diesel' },
            {'label': 'Regular', 'value':'Regular'},
            {'label': 'Super', 'value':'Super'},
            ],
        value = 'Diesel'),
        html.P("Seleccione el tipo de Grafico"),
        dcc.Dropdown(id = 'dropdown_graphs_2',
        options = [
            {'label':'Bar Plot', 'value':'Bar' },
            {'label': 'Box Plot', 'value':'Boxplot'},
            ],
        value = 'Bar'),
    
        html.P("Seleccione el rango de Meses"),
    
        dcc.RangeSlider(id = "range_slider_3",
              min = 1,
              max = len(df),
              step = 20,
              value = [0, len(df)],
              marks = {str(i): str(int(i/21) ) for i in range(0, len(df), 21)}),
        dcc.Graph(id = 'bar_plot_5'),
        dcc.Graph(id = 'bar_plot_6'),
        
        html.P("En las graficas tanto de consumo como de importacion podemos observar un comportamiento muy similar, ya que se mantuvo constante en los mismos meses y con un aumento en Marzo y Diciembre. "),
        html.H2('Modelos de Predicción', style = {'color': 'teal'}),
        html.P("Seleccione el tipo de Modelo"),
    
        dcc.Dropdown(id = 'dropdown_4',
        options = [
            {'label':'Modelo Arima', 'value':'Arima' },
            {'label': 'Modelo Sarimax', 'value':'Sarimax'},
            {'label': 'Modelo Exponencial', 'value':'Exponencial'},
            ],
        value = 'Arima'),
       
        html.P("Seleccione el tipo de Gasolina"),
    
        dcc.Dropdown(id = 'dropdown_fuel',
        options = [
            {'label':'Diesel', 'value':'Diesel' },
            {'label': 'Regular', 'value':'Regular'},
            {'label': 'Super', 'value':'Super'},
            ],
        value = 'Diesel'),
        
        html.P("Seleccione el tipo de linea de tiempo"),
    
        dcc.Dropdown(id = 'dropdown_date',
        options = [
            {'label':'Meses', 'value':'Fechas' },
            {'label': 'Años', 'value':'Anio'},
            ],
        value = 'Fechas'),
        dcc.Graph(id = 'bar_plot_7'),
        dcc.Graph(id = 'bar_plot_8'),
      
        ])


    
    
@app.callback(Output(component_id='bar_plot', component_property= 'figure'),
              Output(component_id='bar_plot_2', component_property= 'figure'),
              Output(component_id='bar_plot_3', component_property= 'figure'),
              Output(component_id='bar_plot_4', component_property= 'figure'),
              Output(component_id='bar_plot_5', component_property= 'figure'),
              Output(component_id='bar_plot_6', component_property= 'figure'),
              Output(component_id='bar_plot_7', component_property= 'figure'),
              Output(component_id='bar_plot_8', component_property= 'figure'),
              Input(component_id='dropdown', component_property= 'value'),
              Input(component_id = "range_slider", component_property = "value"),
              Input(component_id='dropdown_2', component_property= 'value'),
              Input(component_id='dropdown_graphs', component_property= 'value'),
              Input(component_id = "range_slider_2", component_property = "value"),
              Input(component_id='dropdown_3', component_property= 'value'),
              Input(component_id='dropdown_graphs_2', component_property= 'value'),
              Input(component_id = "range_slider_3", component_property = "value"),
              Input(component_id='dropdown_4', component_property= 'value'),
              Input(component_id='dropdown_fuel', component_property= 'value'),
              Input(component_id='dropdown_date', component_property= 'value'))


def graph_update(dropdown_value, rslider_value, drompdown_2_value, dropwdown_graphs_value,rslider_value_2,
                drompdown_3_value, dropwdown_graphs_2_value,rslider_value_3, drompdown_4_value, dropdown_fuel_value, 
                dropdown_date_value):

    
    fig1 = go.Figure([go.Scatter(x = df['Fechas'][rslider_value[0]: rslider_value[len(rslider_value) - 1]], 
                                 y = df['{}'.format(dropdown_value)][rslider_value[0]: rslider_value[len(rslider_value) - 1]],\
                     line = dict(color = '#3591d8', width = 5))])
    
    fig2 = go.Figure([go.Scatter(x = df2['Fechas'][rslider_value[0]: rslider_value[len(rslider_value) - 1]], 
                                 y = df2['{}'.format(dropdown_value)][rslider_value[0]: rslider_value[len(rslider_value) - 1]],\
                     line = dict(color = '#57bd9e', width = 5))])
    
    if dropwdown_graphs_value == "Bar":
    
        fig3 = go.Figure([go.Bar(x = df["Anio"][rslider_value_2[0]: rslider_value_2[len(rslider_value_2) - 1]], 
                                 y = df['{}'.format(drompdown_2_value)])])

        fig4 = go.Figure([go.Bar(x = df2["Anio"][rslider_value_2[0]: rslider_value_2[len(rslider_value_2) - 1]], 
                             y = df2['{}'.format(drompdown_2_value)])])
    else: 
        
        fig3 = go.Figure([go.Box(x = df["Anio"], 
                         y = df['{}'.format(drompdown_2_value)])])

        fig4 = go.Figure([go.Box(x = df2["Anio"], 
                             y = df2['{}'.format(drompdown_2_value)])])
        
    if dropwdown_graphs_2_value == "Bar":
    
        fig5 = go.Figure([go.Bar(x = df["Mes"], 
                                 y = df['{}'.format(drompdown_3_value)])])

        fig6 = go.Figure([go.Bar(x = df2["Mes"], 
                             y = df2['{}'.format(drompdown_3_value)])])
    else:
        
        fig5 = go.Figure([go.Box(x = df["Mes"], 
                                 y = df['{}'.format(drompdown_3_value)])])

        fig6 = go.Figure([go.Box(x = df2["Mes"], 
                             y = df2['{}'.format(drompdown_3_value)])])
        
    if dropdown_date_value == "Fechas":
        
        X = df["Fechas"]
    
        if dropdown_fuel_value == "Diesel":

            fig7 = go.Figure([go.Scatter(x = X,
                             y = DieselCon['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#3591d8', width = 5))])

            fig8 = go.Figure([go.Scatter(x = X,
                                 y = DieselImp['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#57bd9e', width = 5))])

        elif dropdown_fuel_value == "Regular":

            fig7 = go.Figure([go.Scatter(x = X,
                             y = RegularCon['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#256edc', width = 5))])

            fig8 = go.Figure([go.Scatter(x = X,
                                 y = RegularImp['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#009d71', width = 5))])

        else:

            fig7 = go.Figure([go.Scatter(x = X,
                             y = SuperCon['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#88a7d0', width = 5))])

            fig8 = go.Figure([go.Scatter(x = X,
                                 y = SuperImp['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#9ce9de', width = 5))])
    else:
        
        X = df["Anio"].unique()

        if dropdown_fuel_value == "Diesel":
            
            y1 = DieselCon.groupby("Anio").sum()
            y2 = DieselImp.groupby("Anio").sum()
            
            fig7 = go.Figure([go.Scatter(x = X,
                             y = y1['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#3591d8', width = 5))])

            fig8 = go.Figure([go.Scatter(x = X,
                                 y = y2['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#57bd9e', width = 5))])

        elif dropdown_fuel_value == "Regular":

            y1 = RegularCon.groupby("Anio").sum()
            y2 = RegularImp.groupby("Anio").sum()

            fig7 = go.Figure([go.Scatter(x = X,
                             y = y1['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#256edc', width = 5))])

            fig8 = go.Figure([go.Scatter(x = X,
                                 y = y2['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#009d71', width = 5))])

        else:

            y1 = SuperCon.groupby("Anio").sum()
            y2 = SuperImp.groupby("Anio").sum()

            fig7 = go.Figure([go.Scatter(x = X,
                             y = y1['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#88a7d0', width = 5))])

            fig8 = go.Figure([go.Scatter(x = X,
                                 y = y2['{}'.format(drompdown_4_value)],\
                     line = dict(color = '#9ce9de', width = 5))])

    
    fig1.update_layout(title = 'Consumo de gasolina ' + str(dropdown_value),
                      xaxis_title = str(np.abs(rslider_value[len(rslider_value) - 1]  - rslider_value[0])) +' Meses',
                      yaxis_title = 'Cantidad de Barriles Consumidos')
    
    fig2.update_layout(title = 'Importaciones de gasolina ' + str(dropdown_value),
                  xaxis_title = str(np.abs(rslider_value[len(rslider_value) - 1]  - rslider_value[0])) + ' Meses',
                  yaxis_title = 'Importaciones de Barriles Consumidos')
    
    fig3.update_layout(title = 'Consumo Anual de gasolina ' + str(drompdown_2_value),
                  xaxis_title = ' Años',
                  yaxis_title = 'Consumo de Barriles de Gasolina')

    fig4.update_layout(title = 'Importaciones Anual de gasolina ' + str(drompdown_2_value),
              xaxis_title = ' Años',
              yaxis_title = 'Importación de Barriles de Gasolina')
    
    fig5.update_layout(title = 'Consumo Mensual de gasolina ' + str(drompdown_3_value),
                  xaxis_title = ' Meses',
                  yaxis_title = 'Consumo de Barriles de Gasolina')

    fig6.update_layout(title = 'Importaciones Mensual de gasolina ' + str(drompdown_3_value),
              xaxis_title = ' Meses',
              yaxis_title = 'Importación de Barriles de Gasolina')

    if dropdown_fuel_value == "Diesel":

        fig7.update_layout(title = 'Resultados de Consumo de Gasolina ' + dropdown_fuel_value + ' con el modelo ' + str(drompdown_4_value),
                  xaxis_title = ' Meses',
                  yaxis_title = 'Consumo de Barriles de Gasolina')  
    
        fig8.update_layout(title = 'Resultados de Importaciones de Gasolina ' + dropdown_fuel_value + ' con el modelo ' + str(drompdown_4_value),
                  xaxis_title = ' Meses',
                  yaxis_title = 'Importaciones de Barriles de Gasolina')  
        
    elif dropdown_fuel_value == "Regular":
        
        fig7.update_layout(title = 'Resultados de Consumo de Gasolina ' + dropdown_fuel_value + ' con el modelo ' + str(drompdown_4_value),
                  xaxis_title = ' Meses',
                  yaxis_title = 'Consumo de Barriles de Gasolina')  
    
        fig8.update_layout(title = 'Resultados de Importaciones de Gasolina ' + dropdown_fuel_value + ' con el modelo ' + str(drompdown_4_value),
                  xaxis_title = ' Meses',
                  yaxis_title = 'Importaciones de Barriles de Gasolina')  
    
    else: 
        
        fig7.update_layout(title = 'Resultados de Consumo de Gasolina ' + dropdown_fuel_value + ' con el modelo ' + str(drompdown_4_value),
                  xaxis_title = ' Meses',
                  yaxis_title = 'Consumo de Barriles de Gasolina')  
    
        fig8.update_layout(title = 'Resultados de Importaciones de Gasolina ' + dropdown_fuel_value + ' con el modelo ' + str(drompdown_4_value),
                  xaxis_title = ' Meses',
                  yaxis_title = 'Importaciones de Barriles de Gasolina')  
    
    
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8

In [20]:
if __name__ == '__main__': 
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Oct/2021 00:01:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:01:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:01:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:01:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:01:31] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:01:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:01:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:01:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:02:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 00:02:11] "POST /_dash-update-component HTTP/1.1" 200 -
